In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import split, count, when, isnan, col, regexp_replace
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


## Creando una Sesión en Spark 

In [2]:
spark = SparkSession.builder.appName('First Session').getOrCreate()

print('Spark Version: {}'.format(spark.version))

23/10/16 11:15:53 WARN Utils: Your hostname, bltzkrgs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.18 instead (on interface en0)
23/10/16 11:15:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/16 11:15:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Version: 3.5.0


In [3]:
spark

## Cargando los datos (Dataset)

In [5]:
#Defining un esquema

schema = StructType([StructField('mpg', FloatType(), nullable = True),
                     StructField('cylinders', IntegerType(), nullable = True),
                     StructField('displacement', FloatType(), nullable = True),
                     StructField('horsepower', StringType(), nullable = True),
                     StructField('weight', IntegerType(), nullable = True),
                     StructField('acceleration', FloatType(), nullable = True),
                     StructField('model year', IntegerType(), nullable = True),
                     StructField('origin', IntegerType(), nullable = True),
                     StructField('car name', StringType(), nullable = True)])

file_path = 'auto-mpg.csv'

df = spark.read.csv(file_path,
                    header = True,
                    inferSchema = True,
                    nanValue = '?')

df.show(7)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|            car name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|     1|         ford torino|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|     1|    ford galaxie 500|
|14.0|        8|       454.0|     220.0|  4354|         9.0|        70|     1|    chevrolet impala|


## Revisando valores faltantes

<h2>Verificar valores faltantes en un DataFrame de Spark</h2> <p>La función check_missing verifica los valores faltantes (null o NaN) en cada columna de un DataFrame de Spark:</p> <h3>Funcionamiento</h3> <p>La función:</p> <ol> <li>Selecciona todas las columnas del DataFrame que se pasa como parámetro usando <code>dataframe.select(...)</code></li> <li>Recorre cada columna c usando <code>[...] for c in dataframe.columns]</code></li> <li>Evalúa si el valor en c es null o NaN usando <code>when(isnan(c) | col(c).isNull(), c)</code></li> <li>Cuenta los valores donde la condición se cumple usando <code>count(...)</code></li> <li>Asigna un alias a cada cuenta igual al nombre de la columna usando <code>.alias(c)</code></li> <li>Muestra el resultado final usando <code>show()</code></li> </ol> <h3>Ventajas</h3> <ul> <li>Permite verificar rápidamente valores faltantes en todas las columnas</li> <li>Una única función para el chequeo</li> </ul>

In [6]:
def check_missing(dataframe):
    
    return dataframe.select([count(when(isnan(c) | col(c).isNull(), c)). \
                             alias(c) for c in dataframe.columns]).show()

check_missing(df)

+---+---------+------------+----------+------+------------+----------+------+--------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model year|origin|car name|
+---+---------+------------+----------+------+------------+----------+------+--------+
|  0|        0|           0|         6|     0|           0|         0|     0|       0|
+---+---------+------------+----------+------+------------+----------+------+--------+



<h2>Fundamentos de DataFrames en PySpark</h2> <p>Los DataFrames de PySpark almacenan los datos en formato de columnas y filas.</p> <h3>Características</h3> <ul> <li>Cada columna representa alguna característica o variable.</li> <li>Cada fila representa un punto de datos individual.</li> <li>Pueden leer y escribir datos de una variedad de fuentes.</li> </ul> <p>Luego, podemos usar estos DataFrames para aplicar diversas transformaciones sobre los datos.</p> <h3>Uso</h3> <p>Al final de las llamadas de transformación, podemos mostrar u obtener los resultados para visualizarlos o procesarlos.</p> <p>

In [7]:
df.columns

['mpg',
 'cylinders',
 'displacement',
 'horsepower',
 'weight',
 'acceleration',
 'model year',
 'origin',
 'car name']

Mostrarlo en formato pandas

In [8]:
df.toPandas().head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


<h3>Verificar el esquema</h3> <p>La función <code>df.printSchema()</code> imprime el esquema del DataFrame, el cual describe el nombre y tipo de datos de cada columna.</p> <p>Esto nos permite validar que el dataframe cargado tiene la estructura esperada antes de continuar con el procesamiento o análisis de los datos.</p> <h4>Aspectos a revisar</h4> <ul> <li>Nombre y tipo de datos de cada columna</li> <li>Si alguna columna está marcada como nullable</li> <li>Tipos compuestos como struct, array, map, etc.</li> </ul> <p>Verificar el esquema es un paso clave para asegurar la calidad de los datos antes de aplicar transformaciones o construir modelos sobre el dataframe.</p>

In [9]:
#VErficcar el esquema
df.printSchema()

root
 |-- mpg: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: double (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- model year: integer (nullable = true)
 |-- origin: integer (nullable = true)
 |-- car name: string (nullable = true)



<h3>Renombrar columnas</h3> <p>Podemos renombrar columnas en un DataFrame utilizando la función <code>withColumnRenamed()</code>.</p> <h4>Uso</h4> <p>La función <code>withColumnRenamed()</code> toma como parámetros el nombre de la columna original y el nuevo nombre.</p> <p>Esto permite normalizar o formatear mejor los nombres de columnas antes de continuar con el análisis.</p> <h4>Verificación</h4> <p>Al imprimir el DataFrame luego de las renombraciones, podemos verificar que los cambios se aplicaron correctamente.</p> <h4>Beneficios</h4> <p>Renombrar columnas es útil cuando los nombres contienen espacios u otros caracteres que pueden causar problemas al interactuar con el DataFrame.</p>

In [10]:
df = df.withColumnRenamed('model year', 'model_year')

df = df.withColumnRenamed('car name', 'car_name')

df.show(3)



+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|     1|  plymouth satellite|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 3 rows



<h3>Obtener información de las primeras filas</h3> <p>Podemos inspeccionar el contenido de las primeras filas de un DataFrame usando el método <code>head()</code> junto con un bucle <code>for</code>:</p>

<h4>Funcionamiento</h4> <p>El método <code>df.head(4)</code> devuelve las primeras 4 filas del DataFrame como una lista.</p> <p>El bucle <code>for</code> itera sobre esta lista y en cada iteración la variable <code>car</code> toma el valor de una fila.</p> <p>Dentro del bucle, imprimimos cada fila para ver su contenido.</p> <h4>Ventajas</h4> <ul> <li>Permite inspeccionar rápidamente el contenido de las filas</li> <li>Útil para validar estructura y tipo de datos</li> </ul>

In [12]:
for car in df.head(5):
    print(car, '\n')

Row(mpg=18.0, cylinders=8, displacement=307.0, horsepower=130.0, weight=3504, acceleration=12.0, model_year=70, origin=1, car_name='chevrolet chevelle malibu') 

Row(mpg=15.0, cylinders=8, displacement=350.0, horsepower=165.0, weight=3693, acceleration=11.5, model_year=70, origin=1, car_name='buick skylark 320') 

Row(mpg=18.0, cylinders=8, displacement=318.0, horsepower=150.0, weight=3436, acceleration=11.0, model_year=70, origin=1, car_name='plymouth satellite') 

Row(mpg=16.0, cylinders=8, displacement=304.0, horsepower=150.0, weight=3433, acceleration=12.0, model_year=70, origin=1, car_name='amc rebel sst') 

Row(mpg=17.0, cylinders=8, displacement=302.0, horsepower=140.0, weight=3449, acceleration=10.5, model_year=70, origin=1, car_name='ford torino') 



<h3>Resumen estadístico del DataFrame</h3> <p>Podemos obtener un resumen estadístico del contenido de las columnas numéricas en un DataFrame usando el método <code>.describe()</code>.</p> <pre><code>df.describe().show()</code></pre> <h4>Qué genera</h4> <p>El método <code>.describe()</code> genera un nuevo DataFrame que contiene:</p> <ul> <li>Estadísticas básicas como media, desviación estándar, valor mínimo, máximo, etc. por cada columna numérica.</li> <li>Se omiten las columnas de tipo string u objeto.</li> </ul> <h4>Beneficios</h4> <p>Mostrar el resumen estadístico nos permite obtener una visión general rápida sobre:</p> <ul> <li>Rangos de valores.</li> <li>Posibles valores atípicos o erroneos.</li> <li>Distribución de los datos.</li> </ul> <p>Esta información es útil antes de aplicar transformaciones o construir modelos sobre los datos.</p>

In [13]:
df.describe().show()

23/10/16 11:20:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+--------------------+
|summary|               mpg|         cylinders|      displacement|horsepower|           weight|      acceleration|        model_year|            origin|            car_name|
+-------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+--------------------+
|  count|               398|               398|               398|       398|              398|               398|               398|               398|                 398|
|   mean|23.514572864321615| 5.454773869346734|193.42587939698493|       NaN|2970.424623115578|15.568090452261291| 76.01005025125629|1.5728643216080402|                NULL|
| stddev| 7.815984312565783|1.7010042445332123|104.26983817119587|       NaN|846.8417741973268| 2.757688929812676|3.69762664673258

In [14]:
df.describe(['mpg', 'horsepower']).show()

+-------+------------------+----------+
|summary|               mpg|horsepower|
+-------+------------------+----------+
|  count|               398|       398|
|   mean|23.514572864321615|       NaN|
| stddev| 7.815984312565783|       NaN|
|    min|               9.0|      46.0|
|    max|              46.6|       NaN|
+-------+------------------+----------+



In [15]:
#describe con columnas numericas
def get_num_cols(dataframe):
    
    num_cols = [col for col in dataframe.columns if dataframe.select(col). \
                dtypes[0][1] in ['double', 'int']]
    
    return num_cols

num_cols = get_num_cols(df)
    
df.describe(num_cols).show()


+-------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+
|summary|               mpg|         cylinders|      displacement|horsepower|           weight|      acceleration|        model_year|            origin|
+-------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+
|  count|               398|               398|               398|       398|              398|               398|               398|               398|
|   mean|23.514572864321615| 5.454773869346734|193.42587939698493|       NaN|2970.424623115578|15.568090452261291| 76.01005025125629|1.5728643216080402|
| stddev| 7.815984312565783|1.7010042445332123|104.26983817119587|       NaN|846.8417741973268| 2.757688929812676|3.6976266467325862|0.8020548777266148|
|    min|               9.0|                 3|              68.0|      46.0|     

<h2>Operaciones básicas en DataFrame de Spark</h2> <h3>Filtrado y ordenamiento</h3> <p>En esta sección veremos cómo realizar operaciones comunes de filtrado y ordenamiento en DataFrames de Spark:</p> <ul> <li>Filtrar filas que cumplan con una condición sobre una columna</li> <li>Ordenar las filas de acuerdo a una o más columnas</li> <li>Combinar filtros y ordenamiento</li> </ul> <p>Algunos de los métodos que utilizaremos son:</p> <ul> <li>where() para filtrar filas</li> <li>orderBy() para ordenar filas</li> <li>select() para seleccionar columnas</li> </ul> <p>Entendiendo como aplicar estas operaciones básicas será fundamental para explorar, limpiar y preparar los datos antes del análisis.</p> <p>Veremos ejemplos prácticos de cada método y cómo podemos encadenar diferentes transformaciones para lograr el resultado deseado.</p>

In [16]:
# Obtengamos los carro con las mpg mayor a 23
df.filter(df['mpg'] > 23).show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|24.0|        4|       113.0|      95.0|  2372|        15.0|        70|     3|toyota corona mar...|
|27.0|        4|        97.0|      88.0|  2130|        14.5|        70|     3|        datsun pl510|
|26.0|        4|        97.0|      46.0|  1835|        20.5|        70|     2|volkswagen 1131 d...|
|25.0|        4|       110.0|      87.0|  2672|        17.5|        70|     2|         peugeot 504|
|24.0|        4|       107.0|      90.0|  2430|        14.5|        70|     2|         audi 100 ls|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



In [17]:
df.filter((df['horsepower']>80)&(df['weight']>2000)).select('car_name').show(5)

+--------------------+
|            car_name|
+--------------------+
|chevrolet chevell...|
|   buick skylark 320|
|  plymouth satellite|
|       amc rebel sst|
|         ford torino|
+--------------------+
only showing top 5 rows



In [18]:
#Ordenando
df.filter((df['mpg'] > 25) & (df['origin' ] == 2)). \
orderBy('mpg', ascending = False) . show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|44.3|        4|        90.0|      48.0|  2085|        21.7|        80|     2|vw rabbit c (diesel)|
|44.0|        4|        97.0|      52.0|  2130|        24.6|        82|     2|           vw pickup|
|43.4|        4|        90.0|      48.0|  2335|        23.7|        80|     2|  vw dasher (diesel)|
|43.1|        4|        90.0|      48.0|  1985|        21.5|        78|     2|volkswagen rabbit...|
|41.5|        4|        98.0|      76.0|  2144|        14.7|        80|     2|           vw rabbit|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



In [19]:
#Obtener los autos con "volkswagen" en sus nombres, y ordenarlos por año modelo y caballos de fuerza

df.filter(df['car_name'].contains('volkswagen ')). \
orderBy(['model_year', 'horsepower'], ascending=[False, False]) . show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|36.0|        4|       105.0|      74.0|  1980|        15.3|        82|     2| volkswagen rabbit l|
|33.0|        4|       105.0|      74.0|  2190|        14.2|        81|     2|    volkswagen jetta|
|31.5|        4|        89.0|      71.0|  1990|        14.9|        78|     2| volkswagen scirocco|
|43.1|        4|        90.0|      48.0|  1985|        21.5|        78|     2|volkswagen rabbit...|
|29.0|        4|        97.0|      78.0|  1940|        14.5|        77|     2|volkswagen rabbit...|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



In [ ]:
df.filter(df['car_name' ].like( '%volkswagen%' )).show(3)

In [20]:
df_filtrado_ordenado = df.where((col("cylinders") == 6) | (col("cylinders") == 8)).orderBy(col("mpg"), ascending=False)
df_filtrado_ordenado.show()


+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|38.0|        6|       262.0|      85.0|  3015|        17.0|        82|     1|oldsmobile cutlas...|
|32.7|        6|       168.0|     132.0|  2910|        11.4|        80|     3|       datsun 280-zx|
|30.7|        6|       145.0|      76.0|  3160|        19.6|        81|     2|        volvo diesel|
|28.8|        6|       173.0|     115.0|  2595|        11.3|        79|     1|  chevrolet citation|
|26.8|        6|       173.0|     115.0|  2700|        12.9|        79|     1|oldsmobile omega ...|
|26.6|        8|       350.0|     105.0|  3725|        19.0|        81|     1|oldsmobile cutlas...|
|25.4|        6|       168.0|     116.0|  2900|        12.6|        81|     3|     toyota cressida|


In [21]:
df_ordenado = df.orderBy(col("horsepower"), ascending=True).orderBy(col("model_year"), ascending=False)
df_ordenado.show()


+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|28.0|        4|       112.0|      88.0|  2605|        19.6|        82|     1|  chevrolet cavalier|
|38.0|        6|       262.0|      85.0|  3015|        17.0|        82|     1|oldsmobile cutlas...|
|27.0|        4|       112.0|      88.0|  2640|        18.6|        82|     1|chevrolet cavalie...|
|34.0|        4|       112.0|      88.0|  2395|        18.0|        82|     1|chevrolet cavalie...|
|31.0|        4|       112.0|      85.0|  2575|        16.2|        82|     1|pontiac j2000 se ...|
|29.0|        4|       135.0|      84.0|  2525|        16.0|        82|     1|      dodge aries se|
|27.0|        4|       151.0|      90.0|  2735|        18.0|        82|     1|     pontiac phoenix|


## Filtrando con SQL

In [22]:
#Get the cars with 'toyota' in their names
df.filter("car_name like '%toyota%'").show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|24.0|        4|       113.0|      95.0|  2372|        15.0|        70|     3|toyota corona mar...|
|25.0|        4|       113.0|      95.0|  2228|        14.0|        71|     3|       toyota corona|
|31.0|        4|        71.0|      65.0|  1773|        19.0|        71|     3| toyota corolla 1200|
|24.0|        4|       113.0|      95.0|  2278|        15.5|        72|     3|toyota corona har...|
|27.0|        4|        97.0|      88.0|  2100|        16.5|        72|     3|toyota corolla 16...|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



In [23]:
df.filter('mpg > 22').show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|24.0|        4|       113.0|      95.0|  2372|        15.0|        70|     3|toyota corona mar...|
|27.0|        4|        97.0|      88.0|  2130|        14.5|        70|     3|        datsun pl510|
|26.0|        4|        97.0|      46.0|  1835|        20.5|        70|     2|volkswagen 1131 d...|
|25.0|        4|       110.0|      87.0|  2672|        17.5|        70|     2|         peugeot 504|
|24.0|        4|       107.0|      90.0|  2430|        14.5|        70|     2|         audi 100 ls|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



In [24]:
#Multiple Conditions
df.filter('mpg > 22 and acceleration < 15').show(5)

+----+---------+------------+----------+------+------------+----------+------+-------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|     car_name|
+----+---------+------------+----------+------+------------+----------+------+-------------+
|27.0|        4|        97.0|      88.0|  2130|        14.5|        70|     3| datsun pl510|
|24.0|        4|       107.0|      90.0|  2430|        14.5|        70|     2|  audi 100 ls|
|26.0|        4|       121.0|     113.0|  2234|        12.5|        70|     2|     bmw 2002|
|27.0|        4|        97.0|      88.0|  2130|        14.5|        71|     3| datsun pl510|
|25.0|        4|       113.0|      95.0|  2228|        14.0|        71|     3|toyota corona|
+----+---------+------------+----------+------+------------+----------+------+-------------+
only showing top 5 rows



In [25]:
df.filter('horsepower == 88 and weight between 2600 and 3000') \
.select(['horsepower', 'weight', 'car_name']).show()

+----------+------+--------------------+
|horsepower|weight|            car_name|
+----------+------+--------------------+
|      88.0|  2957|         peugeot 504|
|      88.0|  2740|pontiac sunbird c...|
|      88.0|  2720| ford fairmont (man)|
|      88.0|  2890|     ford fairmont 4|
|      88.0|  2870|       ford fairmont|
|      88.0|  2605|  chevrolet cavalier|
|      88.0|  2640|chevrolet cavalie...|
+----------+------+--------------------+



<h3>Registro de DataFrame como vista temporal</h3> <p>El método df.createOrReplaceTempView() se utiliza para registrar el DataFrame como una vista temporal en Spark SQL.</p> <p>Esto permitirá tratar al DataFrame como una tabla y consultarlo usando instrucciones SQL.</p> <h4>Detalles</h4> <p>df es un DataFrame cargado o transformado en Python/PySpark.</p> <p>createOrReplaceTempView() registra el DataFrame con el nombre "autos" como una vista temporal.</p> <p>Las vistas temporales sólo existen durante la sesión actual, se pierden al reiniciar Spark.</p> <p>Al registrarlo como vista, podemos referirnos a él como una tabla y ejecutar queries SQL.</p> <p>Por ejemplo:</p> <pre><code>df.createOrReplaceTempView("autos") spark.sql("SELECT * FROM autos WHERE color='red'")</code></pre>

Filtrar autos con más de 100 caballos de fuerza (horsepower):

In [26]:
df.createOrReplaceTempView("autos")
result = spark.sql("SELECT * FROM autos WHERE horsepower > 100")
result.show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|     1|         ford torino|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



Filtrar autos de origen 1 (EE. UU.):

In [27]:
result = spark.sql("SELECT * FROM autos WHERE origin = 1")
result.show(5)


+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|     1|         ford torino|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



Filtrar autos fabricados antes de 1980:

In [29]:
result = spark.sql("SELECT * FROM autos WHERE `model_year` < 80")
result.show(5)


+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|     1|         ford torino|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



Filtrar autos con un alto consumo de combustible (mpg < 15) y más de 6 cilindros:

In [30]:
result = spark.sql("SELECT * FROM autos WHERE mpg < 15 AND cylinders > 6")
result.show(4)


+----+---------+------------+----------+------+------------+----------+------+------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|          car_name|
+----+---------+------------+----------+------+------------+----------+------+------------------+
|14.0|        8|       454.0|     220.0|  4354|         9.0|        70|     1|  chevrolet impala|
|14.0|        8|       440.0|     215.0|  4312|         8.5|        70|     1| plymouth fury iii|
|14.0|        8|       455.0|     225.0|  4425|        10.0|        70|     1|  pontiac catalina|
|14.0|        8|       340.0|     160.0|  3609|         8.0|        70|     1|plymouth 'cuda 340|
+----+---------+------------+----------+------+------------+----------+------+------------------+
only showing top 4 rows



Filtrar autos de origen 2 (Europa) con una aceleración superior a 20:

In [31]:
result = spark.sql("SELECT * FROM autos WHERE origin = 2 AND acceleration > 20")
result.show(4)


+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|26.0|        4|        97.0|      46.0|  1835|        20.5|        70|     2|volkswagen 1131 d...|
|23.0|        4|        97.0|      54.0|  2254|        23.5|        72|     2|   volkswagen type 3|
|26.0|        4|        97.0|      46.0|  1950|        21.0|        73|     2|volkswagen super ...|
|19.0|        4|       120.0|      88.0|  3270|        21.9|        76|     2|         peugeot 504|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 4 rows



Filtrar autos con mejor relación peso-potencia (horsepower/weight):

In [32]:
result = spark.sql("SELECT * FROM autos WHERE (horsepower / weight) > 0.07")
result.show(4)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|14.0|        8|       455.0|     225.0|  3086|        10.0|        70|     1|buick estate wago...|
|25.0|        4|        98.0|       NaN|  2046|        19.0|        71|     1|          ford pinto|
|21.0|        6|       200.0|       NaN|  2875|        17.0|        74|     1|       ford maverick|
|40.9|        4|        85.0|       NaN|  1835|        17.3|        80|     2|renault lecar deluxe|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 4 rows



Filtrar autos que tengan una aceleración más rápida que el promedio para autos con más de 6 cilindros:

In [33]:
result = spark.sql("SELECT * FROM autos WHERE cylinders > 6 AND acceleration > (SELECT AVG(acceleration) FROM autos WHERE cylinders > 6)")
result.show(4)

+----+---------+------------+----------+------+------------+----------+------+----------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|  car_name|
+----+---------+------------+----------+------+------------+----------+------+----------+
|10.0|        8|       360.0|     215.0|  4615|        14.0|        70|     1| ford f250|
|10.0|        8|       307.0|     200.0|  4376|        15.0|        70|     1| chevy c20|
|11.0|        8|       318.0|     210.0|  4382|        13.5|        70|     1|dodge d200|
| 9.0|        8|       304.0|     193.0|  4732|        18.5|        70|     1|  hi 1200d|
+----+---------+------------+----------+------+------------+----------+------+----------+
only showing top 4 rows



Encontrar el automóvil más potente (con más caballos de fuerza) en cada año modelo:

In [34]:
result = spark.sql("SELECT * FROM autos WHERE (model_year, horsepower) IN (SELECT model_year, MAX(horsepower) FROM autos GROUP BY model_year)")
result.show(4)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|            car_name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|14.0|        8|       455.0|     225.0|  4425|        10.0|        70|     1|    pontiac catalina|
|14.0|        8|       455.0|     225.0|  3086|        10.0|        70|     1|buick estate wago...|
|25.0|        4|        98.0|       NaN|  2046|        19.0|        71|     1|          ford pinto|
|11.0|        8|       429.0|     208.0|  4633|        11.0|        72|     1|     mercury marquis|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 4 rows



Encontrar la media de mpg para autos fabricados antes de 1975 que tienen más de 200 caballos de fuerza:

In [35]:
result = spark.sql("SELECT AVG(mpg) FROM autos WHERE `model_year` < 75 AND horsepower > 200")
result.show(4)

+------------------+
|          avg(mpg)|
+------------------+
|14.583333333333334|
+------------------+



Filtrar autos de origen 3 (Asia) con una relación de desplazamiento por cilindro superior a 50:

In [36]:
result = spark.sql("SELECT * FROM autos WHERE origin = 3 AND (displacement / cylinders) > 50")
result.show()


+---+---------+------------+----------+------+------------+----------+------+--------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|car_name|
+---+---------+------------+----------+------+------------+----------+------+--------+
+---+---------+------------+----------+------+------------+----------+------+--------+



In [37]:
spark.stop()